## Clean Technologies 

In [1]:
import pandas as pd

In [2]:
categories = pd.read_csv('../data/categories.csv', sep='\t')

In [3]:
categories.columns = ['sector','group','technology']

In [4]:
cordish2020 = pd.read_excel('../data/cordis-h2020projects.xlsx')

In [5]:
cordish2020.head()

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,objective,totalCost,ecMaxContribution,call,fundingScheme,coordinator,coordinatorCountry,participants,participantCountries,subjects
0,196618,643555,UNCAP,CLOSED,H2020-EU.3.1.6.,PHC-20-2014,H2020,Ubiquitous iNteroperable Care for Ageing People,2015-01-01,2017-12-31,...,The lack of ICT platforms based on open standa...,3820763.91,3086254.00,H2020-PHC-2014-single-stage,/IA,TRILOGIS SRL,IT,CREATE-NET (CENTER FOR RESEARCH AND TELECOMMUN...,IT;RO;DE;SE;EL;ES;MK;UK;SI,NaN
1,220199,822166,Governance,SIGNED,H2020-EU.3.6.3.;H2020-EU.3.6.1.,GOVERNANCE-14-2018,H2020,Democratic governance in a turbulent age,2018-12-01,2023-11-30,...,Democratic politics and governance in Europe a...,9090927.00,3000000.00,H2020-SC6-GOVERNANCE-2018,/ERA-NET-Cofund,NEDERLANDSE ORGANISATIE VOOR WETENSCHAPPELIJK ...,NL,STYRELSEN FOR FORSKNING OG UDDANNELSE;FONDS ZU...,DK;AT;EE;IS;LT;DE;SE;SI;CZ;ES;FI;PL;UK;NO;CH;I...,NaN
2,220196,822189,BRaVE,SIGNED,H2020-EU.3.6.1.2.,SU-GOVERNANCE-11-2018,H2020,Building Resilience Against Violent Extremism ...,2019-01-01,2021-06-30,...,Far Right and Islamist groups seeking to recru...,1483750.00,1483750.00,H2020-SC6-GOVERNANCE-2018,/CSA,EUROPEAN UNIVERSITY INSTITUTE,IT,DUBLIN CITY UNIVERSITY;KOZEP-EUROPAI EGYETEM;C...,IE;HU;BE;UK;PL;DE,NaN
3,216332,801338,VES4US,SIGNED,H2020-EU.1.2.1.,FETOPEN-01-2016-2017,H2020,Extracellular vesicles from a natural source f...,2018-09-01,2021-08-31,...,"Extracellular vesicles (EVs) are cell-derived,...",2946303.75,2946303.75,H2020-FETOPEN-1-2016-2017,/RIA,CONSIGLIO NAZIONALE DELLE RICERCHE,IT,"ZABALA INNOVATION CONSULTING, S.A.;INSTITUTE O...",ES;IE;CH;DE;SI,NaN
4,216023,770045,DIALLS,SIGNED,H2020-EU.3.6.3.2.,CULT-COOP-03-2017,H2020,DIalogue and Argumentation for cultural Litera...,2018-05-01,2021-04-30,...,DIALLS is a three year project with three obje...,4373308.75,4373308.75,H2020-SC6-CULT-COOP-2017-two-stage,/RIA,THE CHANCELLOR MASTERS AND SCHOLARS OF THE UNI...,UK,EDEX - EDUCATIONAL EXCELLENCE CORPORATION LIMI...,CY;FI;PT;LT;ES;IL;DE;FR,NaN


#### Flashtext for easy regex

In [6]:
from flashtext import KeywordProcessor

In [7]:
def extract(vec, dictionary, info=False):
    empty = []
    for line in vec:
        empty.append(dictionary.extract_keywords(line, span_info=info))
    return empty

In [8]:
for column in categories.columns:
    dictionary = KeywordProcessor()
    dictionary.add_keywords_from_list(list(categories[str(column)]))
    extracted = extract(cordish2020['objective'], dictionary)
    #cordish2020['matches_'+str(column)] = extracted
    cordish2020['matches_'+str(column)] = [list(set(i)) if len(i)>0 else '' for i in extracted]
    cordish2020['count_matches_'+str(column)] = [len(i) for i in extracted]
    cordish2020['count_unique_matches_'+str(column)] = [len(set(i)) for i in extracted]

In [9]:
cordish2020['count_matches'] = cordish2020['count_matches_sector'] + cordish2020['count_matches_group'] + cordish2020['count_matches_technology']

In [10]:
cordish2020matches = cordish2020[cordish2020['count_matches'] > 0 ]

In [11]:
print(len(cordish2020matches)/len(cordish2020))

0.3882076827282307


In [12]:
cordish2020matches.sort_values('count_matches', ascending=False).head()

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,matches_sector,count_matches_sector,count_unique_matches_sector,matches_group,count_matches_group,count_unique_matches_group,matches_technology,count_matches_technology,count_unique_matches_technology,count_matches
20293,194405,645963,SENSIBLE,SIGNED,H2020-EU.3.3.,LCE-08-2014,H2020,Storage-Enabled Sustainable Energy for Buildin...,2015-01-01,2018-06-30,...,"[Buildings, Energy]",20,2,"[Energy management, Storage]",13,2,"[energy storage, heat pumps, thermal storage]",4,3,37
14652,210955,739615,acteno,SIGNED,H2020-EU.2.3.2.2.,INNOSUP-02-2016,H2020,Industrial Energy Storage Integration - develo...,2017-09-01,2018-08-31,...,"[Industry, Energy]",12,2,[Storage],14,1,[energy storage],4,1,30
1922,218681,824066,E-MAGIC,SIGNED,H2020-EU.1.2.2.,FETPROACT-01-2018,H2020,European Magnesium Interactive Battery Community,2019-01-01,2022-12-31,...,"[Transport, Buildings, Energy]",19,3,[Storage],6,1,[energy storage],5,1,30
18909,200258,691799,UPWAVE,TERMINATED,H2020-EU.3.3.;H2020-EU.3.;H2020-EU.3.3.2.4.;H2...,LCE-03-2015,H2020,Demonstration of a 1-MW wave energy converter ...,2016-02-01,2021-01-31,...,"[Industry, Energy]",11,2,[Wind],8,1,"[offshore wind, wave energy]",9,2,28
13310,213876,808585,gridBox,SIGNED,H2020-EU.3.3.;H2020-EU.2.1.1.;H2020-EU.2.3.1.,SMEInst-09-2016-2017,H2020,A smart energy management system for energy tr...,2018-01-01,2018-04-30,...,[Energy],21,1,"[Energy management, Storage]",4,2,[energy storage],2,1,27


There is one hypthesis here:  
The more count matches you have the better the accuracy to detect technology terms in the objective text  
Might be solved with statistical sampling. For each count match, select a sample check manually the accuracy  
http://www.marknagelberg.com/using-python-to-figure-out-sample-sizes-for-your-study/

EDA
1. Group by month and count over the time
2. Group by month and sum the total cost over the time
3. Group by country coordinator and count

In [13]:
def get_terms_matrix(vector_of_terms, all_terms):
    matrix = []
    for vector in vector_of_terms:
        ind_vector = [0] * len(all_terms)
        for v in vector:
            for idx, i in enumerate(all_terms):
                if v == i:
                    ind_vector[idx] = 1
        matrix.append(ind_vector)
    table = pd.DataFrame(matrix)
    return table

In [14]:
all_terms = [i for i in set([*list(categories['sector']), *list(categories['group']), *list(categories['technology'])])]

In [15]:
matches_technology_table = get_terms_matrix(list(cordish2020matches['matches_technology']), all_terms)
matches_group_table = get_terms_matrix(list(cordish2020matches['matches_group']), all_terms)
matches_sector_table = get_terms_matrix(list(cordish2020matches['matches_sector']), all_terms)

In [16]:
matches_table = matches_sector_table + matches_group_table + matches_technology_table

In [17]:
matches_table.columns = all_terms

In [18]:
matches_table['count'] = matches_table.sum(axis=1)

In [19]:
matches_table.head()

,Wind,Building design,low GHG transmission,System innovation,passive house,Transport,Urban forestry,low GHG air conditioning,urban forestry,Energy,...,smelt reduction,reuse of material,geothermal,biofuel,methane reduction,grid management,Vehicle design,compressed air energy storage (CAES),waste management,count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
len(matches_table)

8105

In [21]:
full_table_tech_matches = pd.concat([cordish2020matches.reset_index(), matches_table], axis=1).sort_values(['count_matches','count'], ascending=False).reset_index()

In [22]:
len(full_table_tech_matches)

8105

In [23]:
full_table_tech_matches[['id','acronym','title','objective','count','count_matches']].head(30)

,id,acronym,title,objective,count,count_matches
0,645963,SENSIBLE,Storage-Enabled Sustainable Energy for Buildin...,The project SENSIBLE addresses the call LCE-08...,7,37
1,824066,E-MAGIC,European Magnesium Interactive Battery Community,Energy storage is a key technology to facilita...,5,30
2,739615,acteno,Industrial Energy Storage Integration - develo...,We see industrial grade storage solutions as t...,4,30
3,691799,UPWAVE,Demonstration of a 1-MW wave energy converter ...,The aim is to develop and install a pre-commer...,5,28
4,816706,EOLI FPS,Roof top wind turbine for urban areas,High electricity prices and the lowering costs...,5,27
5,808585,gridBox,A smart energy management system for energy tr...,Imbalances in the energy generation and demand...,4,27
6,673782,FLOATMAST,An Innovative Wind Resource Assessment Tension...,The proposal is about the development of an in...,4,26
7,698136,WITRO,Wind and Turbulence Radar for Offshore wind en...,The project studies the use of a radar based t...,3,26
8,691761,CryoHub,Developing Cryogenic Energy Storage at Refrige...,The CryoHub innovation project will investigat...,3,24
9,726776,VORTEX,NEW CONCEPT OF AFFORDABLE WIND ENERGY GENERATO...,We have developed a resource-efficient and aff...,3,23


In [37]:
termsT = full_table_tech_matches.iloc[:,33:-1].T
termsT['sum'] = termsT.sum(axis=1)

In [38]:
termsT.sort_values('sum', ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,8096,8097,8098,8099,8100,8101,8102,8103,8104,sum
Energy,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,3542
Industry,0,0,1,1,0,0,1,0,0,1,...,0,1,1,0,0,1,1,1,1,3514
Transport,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1218
Storage,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,784
Fuel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,686
Agriculture,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,423
Buildings,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,421
Wind,0,0,0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,344
recycling,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,251
energy storage,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,194


In [39]:
len(termsT[termsT['sum']>0])/len(termsT)

0.5514705882352942